# Cleans dummy data for xml writing

In [ ]:
import pandas as pd
import os 

#finaldf = []

#for d in newlist:
    #df = pd.read_excel(d,sheet_name="Directory",index_col=0)
    #dirdict = df.to_dict()
    #dirdict
    #directory = dirdict['Edge ID']
    

def cleaning(df):
    
    def time_convert(df):
        #Seperates Time column to 'Begin' and 'End'
        df["Begin"] = df['Time'].apply(lambda x: x.split(" - ")[0])
        df["End"] = df['Time'].apply(lambda x: x.split(" - ")[1])
        df = df.drop(columns=['Time'])

        #Puts 'Begin' and 'End' in the front of the dataframe
        reorganized_columns = df.columns[-2:].to_list() + df.columns[0:len(df.columns)-2].to_list()
        df = df[reorganized_columns]

        #fill null values with 0
        df.fillna(0, inplace=True)

        #Change all instances of 12:00nn to correct 12:00pm
        df.loc[df['Begin'] == '12:00nn','Begin'] = '12:00pm'
        df.loc[df['End'] == '12:00nn','End'] = '12:00pm'

        #Create a function that converts string time to seconds 
        def time_to_seconds(val):
            val = pd.to_datetime(val).to_pydatetime()
            seconds = val.timestamp()
            return int(seconds)

        #applies previous function
        df["Begin"] = df['Begin'].apply(time_to_seconds)
        df["End"] = df['End'].apply(time_to_seconds)

        #Modify time to start at 0
        time_start = df["Begin"][0]
        df["Begin"] = df["Begin"] - time_start
        df["End"] = df["End"] - time_start

        all_df = []
        
        return df

    newdf = time_convert(df)

    df2 = newdf.drop(columns=['Begin','End','From','To','Total'])

    begin_val = []
    end_val = []
    vehicleclass_val = []   
    from_val = []
    to_val = []
    count_val = []

    #use initial df for rows, and df3 for columns

    for idx_r, row in enumerate(newdf.index.to_list()):
        for idx_c, column in enumerate(df2.columns.to_list()):
            from_val.append(df.iloc[idx_r,12])
            to_val.append(df.iloc[idx_r,13])
            begin_val.append(newdf.iloc[idx_r,0])
            end_val.append(newdf.iloc[idx_r,1])

    for idx_r, row in enumerate(df2.index.to_list()):
        for idx_c, column in enumerate(df2.columns.to_list()):        
            vehicleclass_val.append(df2.columns.to_list()[idx_c])
            count_val.append(df2.iloc[idx_r, idx_c])


    new_df = pd.DataFrame({'Begin': begin_val,'End': end_val,'class':vehicleclass_val,'Count':count_val,'From':from_val,'To':to_val})
    new_df.fillna(0,inplace=True)

    
    
    #df3.to_csv("finaldata.csv")
    return new_df


def replace_dir(new_df, directory1):
    df3 = new_df.replace({"From":directory1}) 
    return df3
    
def replace_dir2(new_df, directory2):
    df4 = new_df.replace({"To":directory2}) 
    return df4    


In [11]:
import pandas as pd
import os
from pathlib import Path

folderpath = Path(r"Preprocessed Data")                                             # Raw Data
route_path = "Simulation_Environment\Final Test Envi\Route Files"                   # Turn File
final_route_path = "Simulation_Environment\Final Test Envi\Final Route Folder"      # Conversion to route xnl output folder


net_file = "Simulation_Environment\Final Test Envi\osm.net.xml"                     # Net File of Environment

newlist=[]
def listDir(dir):
    fileNames = os.listdir(dir)
    for fileName in fileNames:
       newlist.append(os.path.abspath(os.path.join(dir, fileName)))

if __name__ == '__main__':
    listDir(folderpath)

newlist


['d:\\Main GDrive\\Programming\\Gthub Repo\\DQL_AveEntLumRei\\Preprocessed Data\\Capt.-Vicente-Roa_Gen.-Luna-FINAL-NAJUD.xlsx',
 'd:\\Main GDrive\\Programming\\Gthub Repo\\DQL_AveEntLumRei\\Preprocessed Data\\Claro-M.-Recto_Capt.-Vicente-Roa-FINAL-NAJUD.xlsx',
 'd:\\Main GDrive\\Programming\\Gthub Repo\\DQL_AveEntLumRei\\Preprocessed Data\\Claro-M.-Recto_Corrales-Ave.-FINAL-NAJUD.xlsx',
 'd:\\Main GDrive\\Programming\\Gthub Repo\\DQL_AveEntLumRei\\Preprocessed Data\\Claro-M.-Recto_Osmena-FINAL-NAJUD.xlsx',
 'd:\\Main GDrive\\Programming\\Gthub Repo\\DQL_AveEntLumRei\\Preprocessed Data\\Gen.-Luna_Corrales-Ave.-FINAL-NAJUD.xlsx',
 'd:\\Main GDrive\\Programming\\Gthub Repo\\DQL_AveEntLumRei\\Preprocessed Data\\Osmena_-Florentino-FINAL-NAJUD.xlsx',
 'd:\\Main GDrive\\Programming\\Gthub Repo\\DQL_AveEntLumRei\\Preprocessed Data\\Osmena_-Gen.-Luna-FINAL-NAJUD.xlsx']

In [5]:
all_excel = newlist
all_df =[]

for sheet in all_excel:
    df = pd.read_excel(sheet, sheet_name="TVC")
    df = cleaning(df)
    dir_dict = pd.read_excel(sheet, sheet_name="Directory1", index_col=0).to_dict()['Edge ID']
    dir_dict2 = pd.read_excel(sheet, sheet_name="Directory2", index_col=0).to_dict()['Edge ID']

    df = replace_dir(df, dir_dict)
    df = replace_dir2(df, dir_dict2)
    all_df.append(df)

all_tls_df = pd.concat(all_df)
all_tls_df = all_tls_df.sort_values(by='Begin').reset_index(drop=True).round(decimals=0)
all_tls_df.to_csv('FINALDUMMY.csv')
all_tls_df


,Begin,End,class,Count,From,To
0,0,3600,cars_SUV,6.0,east_luna_0,south_roa_1
1,0,3600,bicycle_tricycle,2.0,west_luna_1,north_roa_1
2,0,3600,largeTrucks,2.0,west_luna_1,north_roa_1
3,0,3600,smallTrucks,10.0,west_luna_1,north_roa_1
4,0,3600,buses,0.0,west_luna_1,north_roa_1
...,...,...,...,...,...,...
8545,50400,54000,largeTrucks,10.0,south_osmena_1,west_florentino_0
8546,50400,54000,smallTrucks,64.0,south_osmena_1,west_florentino_0
8547,50400,54000,buses,6.0,south_osmena_1,west_florentino_0
8548,50400,54000,motorela,4.0,south_roa_0,east_luna_1


## The code below writes the data into XML format

In [8]:
import pandas as pd
import os

df = pd.read_csv("FINALDUMMY.csv", index_col=0)

def df_to_xml(df):
    xml_by_class = {}
    sort_vehicle_class = [i for _, i in df.groupby('class')]
    for vehicle_df in sort_vehicle_class:
        sort_time = [i for i in vehicle_df.groupby('Begin')]
        body_message = []
        for sample in sort_time:
            sample_df = sample[1]
            sample_df = sample_df.reset_index(drop=True)
            starting_message=' <interval id="begin_{}" begin="{}" end="{}">\n'.format(sample_df['Begin'][0],sample_df['Begin'][0],sample_df['End'][0])
            body_message.append(starting_message)
            for idx in range(sample_df.shape[0]):
                _,_,vehicle_type,count,from_edge,to_edge= sample_df.iloc[idx,:]
                if count == 0:
                    continue
                message = '  <edgeRelation from="{}" to="{}" count="{}"/>\n'.format(from_edge,to_edge,count)
                body_message.append(message)
            body_message.append(' </interval>\n')
        xml_by_class[vehicle_type] = body_message

    for key, val in xml_by_class.items():
        filename = f"{key}.xml"
        start = "<data>\n"
        end = "</data>"

        with open(Path(f"{route_path}\\{filename}"), "w") as f:
            f.write(start)
            for line in val:
                f.write(line)
            f.write(end)
    print("Sorted Done")
    
df_to_xml(df)

Sorted Done


### Write Script

In [30]:
import os
import subprocess as sp

route_files = os.listdir(route_path)

t_files = []
vtypes = []

#Gets the vtype and t_files path
for f in route_files:
    vtype = f.split('.')[0]
    t_file = os.path.abspath(os.path.join(route_path,f))
    
    vtypes.append(vtype)
    t_files.append(t_file)
#Run jtcrouter
for vtype, t_file in zip(vtypes, t_files):
    # os.system(f'python "C:\Program Files (x86)\Eclipse\Sumo\tools\jtcrouter.py" -n {net_file} -t "{t_file}" -o "{os.path.join(final_route_path, "vtype_route.xml")}" --fringe-flows --prefix {vtype[0:3]}')
    # sp.run(f'python "C:\Program Files (x86)\Eclipse\Sumo\tools\jtcrouter.py" -n {net_file} -t "{t_file}" -o "{os.path.join(final_route_path, f"{vtype}_route.xml")}" --fringe-flows --prefix {vtype[0:3]}', shell=True)
    sp.run(['python' ,'C:\Program Files (x86)\Eclipse\Sumo\tools\jtcrouter.py', "-n", net_file, "-t", t_file, os.path.join(os.path.abspath(final_route_path), f"{vtype}_route.xml"), "--fringe-flows", "--prefix", vtype[0:3]])

In [1]:
import subprocess as sp

sp.call(["python", "-v"])

In [ ]:
python "C:\Program Files (x86)\Eclipse\Sumo\tools\jtcrouter.py" -n "D:\School Documents\Thesis\DQL_AveEntLumRei\Simulation_Environment\Main MADDPG\osm.net.xml" -t "Route Files\smallTrucks.xml" -o "Simulation_Environment\Main MADDPG\Final Route Folder\smallTrucks_route.xml" --fringe-flows --prefix i